# Floresta de Caminhos Ótimos (Optimum-Path Forest)

O OPF é um *framework* para o desenvolvimento de classificadores *supervisionados* e *não-supervisionados* (a versão supervisionada foi criada pelo professor *João Paulo Papa*). A base destes métodos é a representação dos conjuntos de amostras como grafos.

## OPF Supervisionado

O OPF supervisionado é utilizado para tarefas de classificação, como já estamos acostumados a fazer. Basicamente, o treinamento desse modelo se divide nas seguintes etapas:

### 1 - Representação dos dados

O conjunto de dados é representado por um grafo, onde cada vértice (ou nó) representa uma amostra e sua posição no grafo é representado pelo valor de cada uma de suas características. Esses vértices são conectados por arestas, cujos pesos são representados pela distância entre o par de vértices. 


<img src='assets/graphOPF.jpeg' width=400px>

Note que este grafo pode ser completo, i.e., todos os vértices são conectados a todos os outros vértices, ou $k$-nn, ou seja, cada vértice está conectado aos $k$ vizinhos mais próximos.

- Grafo Completo
<img src='assets/grafoCompleto.jpeg' width=400px>

- Grafo $k$-nn
<img src='assets/grafoknn.jpeg' width=400px>

### 2 - Obtenção dos protótipos

Os protótipos são as amostras mais representativas de cada classe, e iniciam o processo de conquista das demais amostras. Esses protótipos podem ser encontrados em um processo de 2 etapas:

1. Computar a árvore geradora mínima ( _Minimum spanning tree (MST)_ ). Essa árvore pode ser computada utilizando diversos algoritmos, como por exemplo Dijkstra, Prim ou Kruskal. Geralmente Dijkstra é o utilizado. 

<img src='assets/Dijkstra_Animation.gif' width=400px>

2. Seleciona os vértices na região de fronteira entre as classes como protótipos:

<img src='assets/prototipos.png' width=400px>


### 3 - Conquista e classificação

Após a definição dos protótipos, cada amostra recebe um custo associado, definido da seguinte maneira:

\begin{eqnarray}
 \label{e.fmax}
f_{max}(\langle
q\rangle) & = & \left\{ \begin{array}{ll}
  0 & \mbox{if q $\in {\cal P}$,} \\
  +\infty & \mbox{otherwise}
  \end{array}\right. \nonumber \\
  f_{max}(\phi_{q} \cdot \langle q,u \rangle) & = & \max\{f_{max}(\phi_{q}),d(q,u)\}, 
\end{eqnarray}


onde ${\cal P}$ é o conjunto de protótipos, $\phi_{q}$ é um caminho, i.e., uma sequência de nós adjacentes iniciado em ${\cal P}$ e terminando em ${q}$, e $d(q,u)$ entre os nós $q$ e $u$. Além disso, $\phi_{q} \cdot \langle q,u \rangle$ é a concatenação entre o caminho $\phi_{q}$ e a aresta $\langle q,u \rangle$. Resumindo, $f_{max}(\phi_{q})$ é a distância máxima de todas as amostras no caminho $\phi_{q}$.

Na sequência, é estabelecida uma competição entre os protótipos para definir a conquista das damais amostras. O processo é executado de forma a minimizar o custo associado a cada amostra, da seguinte maneira:

\begin{equation}
\label{e.conquering_function}
	C(u)  =  \min_{\forall q \in {\cal V}}\{\max\{C(q),d(q,u)\}\}, 
\end{equation}
onde ${\cal V}$ é o conjunto de vizinhos de $u$.

## Classificação com OPF utilizando OPFython

O código para executar o OPF está disponível em Python atravéz do pacote [OPFython](https://arxiv.org/abs/2001.10420), que pode ser clonado do repositório [Github](https://github.com/gugarosa/opfython) ou instalado via pip com o comando `pip install opfython`.

In [18]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

# importando função para dividir o conjunto em treinamento e teste, assim como 
#   fazemos com a função train_test_split do scikit-learn
import opfython.stream.splitter as s

# Importando o OPF supervisionado para classificação
from opfython.models.supervised import SupervisedOPF


# usando as configurações padrões do seaborn para plotagem
import seaborn as sns; sns.set()

In [45]:
from sklearn.datasets import make_circles

X, y = make_circles(200, factor=.1, noise=.1)

plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='autumn')

In [21]:
# Dividindo os dados em treinamento e teste
X_train, X_test, Y_train, Y_test = s.split(
    X, y, percentage=0.8, random_state=1)


# Cria uma instancia do OPF supervisionado
opf = SupervisedOPF()

# Ajusta o classificador considerando o conjunto de treinamento
opf.fit(X_train, Y_train)

# Predição no conjunto de testes
preds = opf.predict(X_test)

In [46]:
from sklearn.metrics import classification_report
print(classification_report(Y_test, preds))

## Aprendizado não supervisionado

Em problemas de aprendizado não-supervisionado, temos apenas os dados de entrada ($X$) sem rótulos, e gostariamos que o algoritmo seja capaz de encontras alguma estrutura que represente os dados. Um algoritmo de agrupamento, como o OPF não-supervisionado, por exemplo, tenta juntar amostras em **agrupamentos** ( _clusters_ ) de acordo com alguma similaridade.


Exemplos: Análise de redes sociais, organização de _clusters_ de computadores, análise de dados astronômicos.





### Clustering (agrupamento)

Dado um conjunto de pontos, a ideia é agrupá-los de modo que:

- amostras dentro de cada agrupamento são similares entre si;
- amostras de agrupamentos diferentes são dissimilares;

Similaridade é definida usando alguma métrica de distância:

- euclidiana;
- cossenos;
- mahalanobis ...


<img src='assets/clustering1.jpg' width=900px>


## OPF Não-supervisionado

De forma similar a versão supervisionada, na versão não-supervisionada as amostras também são representadas como nós em um grafo, são escolhidos os protótipos, seguido pela etapa de conquista. No entanto, existem algumas diferenças:

### Obtenção dos protótipos

Na versão não-supervisionada, o peso inicial de cada amostra é dado pelo valor de sua densidade, a qual é computada usando a seguinte função de distribuição de probabilidade:

\begin{eqnarray}
  \rho(q) & = & \frac{1}{\sqrt{2\pi\psi^2}k} \sum_{\forall u\in {\cal A}_k(q)} \exp\left(\frac{-d^2(q,u)}{2\psi^2}\right), \label{e.density}
\end{eqnarray}

onde ${\cal A}_k(q)$ representa os $k$ vizinhos mais próximos da amostra $q$, $\psi = \frac{d_f}{3}$, e $d_f$ é o peso máximo entre as arestas no grafo. Note que o número de vizinhos ideal $k^*$ é escolhido de acordo com uma métrica de corte no grafo, onde todos os valores de $k$ no intervalo [$k_{min}$,$k_{max}$] ($k_{min}$ e $k_{max}$ são hiperparâmetros) são testados em uma busca axaustiva.

Todas as amostras são adicionadas em uma lista ordenada pelo valor de suas respectivas densidades de forma decrescente. A primeira amostra (com maior densidade) é removida dessa lista e automáticamente se transforma em um protótipo e tenta conquistar as amostras vizinhas. As proximas amostras são removidas consecutivamente da lista e é verificado se essa amostra já foi conquistada. Caso não tenha sido conquistada, ela também se transforma em um protótipo.

## Conquista

Para conquistar uma amostra durante o treinamento, um protótipo ou uma amostra conquistada deve oferecer um custo melhor ao que essa amostra já possui. Esse custo é computado da seguinte forma:


\begin{eqnarray}
\label{e.pf2}
f_{min}(\langle u \rangle) & = & \left\{ \begin{array}{ll} 
    \rho(u)           & \mbox{if $u \in {\cal P}$} \\
    \rho(u) - \delta  & \mbox{otherwise,}
 \end{array}\right. \nonumber \\
f_{min}(\langle \phi_q\cdot \langle q,u\rangle\rangle)&=& \min \{f_{min}(\phi_{q}), \rho(u)\},
\end{eqnarray}
onde $\delta$ é uma constante pequena. A Figura abaixo ilusta o procedimento:

<img src='assets/naoSup.jpeg' width=600px>

- (a) densidades
- (b) removeu 1° amostra, que se tornou protótipo
- (c) após processo de conquista

## Clustering com OPF utilizando OPFython

In [23]:
import opfython.math.general as g
import opfython.stream.parser as p
from opfython.models import UnsupervisedOPF

from sklearn.datasets import make_blobs

# Para não ficar mostrando um monte de logs
import logging
logging.disable()

In [47]:
X, y_true = make_blobs(n_samples=300, centers=4,
                       cluster_std=0.60, random_state=0)
plt.scatter(X[:, 0], X[:, 1], s=50);

In [40]:
# Cria uma instância do opf não supervisionado
opf = UnsupervisedOPF(
    min_k=1, max_k=20)

In [41]:
# Ajusta o modelo aos dados
opf.fit(X, y_true)

In [42]:
# Se os dados estiverem rotulados, propaga o rótulo real de cada protótipo para todas as amostras por ele
#   conquistadas
opf.propagate_labels()

In [48]:
# faz a estimativa dos rótulos baseado no agrupamento
preds, clusters = opf.predict(X)


print('Número de agrupamentos = ',len(np.unique(clusters)))

plt.scatter(X[:, 0], X[:, 1], c=clusters, s=50, cmap='rainbow')


## Classificando com os rótulos propagados

Caso tenha propagado os rótulos, é possível utilizar essa abordagem para executar tarefas de classificação:

In [49]:
print(classification_report(y_true, preds))

# Exercícios:

Baixe os datasets Breast Tissue e Landsat Satellite do repositório UCI.

1. Separar em treinamento e teste e classificar as amostras de teste utilizando o OPF supervisionado. 

2. Agrupar os dados utilizando o OPF-não supervisionado e propagar o rótulo considerando um conjunto de treinamento. Classificar as amostras de teste considerando os rótulos propagados.